In [1]:
import os
import sys
import torch
import datetime
import numpy as np
from utils import *
from model import *
from tqdm import tqdm
from time import time
from torch import Tensor, nn
from dataset_factory import *
import torch_optimizer as optim
import torch.distributed as dist
from dataclasses import dataclass
from accelerate import Accelerator
from transformers.file_utils import ModelOutput
from torch.utils.data import DataLoader, Dataset
from typing import List, Optional, Tuple, Union, Dict
from deepspeed.ops.adam import DeepSpeedCPUAdam, FusedAdam
from torch.optim.lr_scheduler import CosineAnnealingLR, LinearLR
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel
os.environ['http_proxy'] = 'http://172.19.57.45:3128'
os.environ['https_proxy'] = 'http://172.19.57.45:3128'
sys.path.append('/root/paddlejob/workspace/env_run/ReGPT')
%cd /root/paddlejob/workspace/env_run/ReGPT
config = get_config('config/rellama_config.yaml')
ReGPT_kwargs = config['ReGPT_kwargs']
config['training'].update(ReGPT_kwargs)
config['dataset']['train'].update(ReGPT_kwargs)
config['dataset']['test'].update(ReGPT_kwargs)
generation_kwargs = config['generation_kwargs']
config['training'].update(generation_kwargs)
train_config = config['training']
dataset_config = config['dataset']
train_config['negative_depth'] = dataset_config['train']['negative_depth']
dataset_config['train']['predict_from_last'] = train_config['predict_from_last']
dataset_config['test']['predict_from_last'] = train_config['predict_from_last']

tokenizer = AutoTokenizer.from_pretrained(train_config['tokenizer_name_or_path'])
tokenizer.pad_token = tokenizer.eos_token
train_config['eos_token_id'] = tokenizer.eos_token_id

dataset_class = {"DialogSFTDataset": DialogSFTDataset, "CorpusPretrainDataset": CorpusPretrainDataset, "ReGPTDialogSFTDataset": ReGPTDialogSFTDataset, "ReGPTCorpusPretrainDataset": ReGPTCorpusPretrainDataset}

config['training']['model_name_or_path'] = 'output/SFT-best/'
print_args(config)
model = ReGPTForCausalLM(train_config)
# model.cuda()
# model = model.half()


def move_to_cuda(kwargs, device='cuda:0'):
    for key in kwargs:
        kwargs[key] = kwargs[key].to(device)
def generate(text='Tsinghua University is a '):
    text+=' '
    kwargs = tokenizer([text],return_tensors='pt')
#     move_to_cuda(kwargs)
    output_ids = model.generate(**kwargs)
#     print(tokenizer.convert_ids_to_tokens(output_ids.cpu().numpy().squeeze()))
    return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(output_ids.cpu().numpy().squeeze()))

from time import time
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('Beethoven'))
print(time()-start)

[2023-12-10 22:17:25,864] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/root/paddlejob/workspace/env_run/ReGPT
trainer: ReGPTLanguageModelTrainer
training:
	project_name: llama2-7b-sft
	model_name_or_path: output/SFT-best/
	tokenizer_name_or_path: ../data_of_ReGPT/llama2-7b-phrase-tokenizer-trained-on-WikiText103/
	num_layers_unfrozen: -1
	gradient_checkpointing: True
	optimizer:
		name: Lamb
		kwargs:
			lr: 1e-4
			weight_decay: 0.0
			betas: [0.9, 0.999]
			eps: 1e-08
	scheduler:
		name: CosineAnnealingLR
		kwargs:
			T_max: 1000
			eta_min: 3e-5
	num_epochs: 1000
	log_with: tensorboard
	project_dir: output/
	negatives_in_device: True
	predict_from_last: 10000
	eval_step: 1000
	faiss:
		index_type: Flat
		dimension: 768
		nprobe: 10000
		phrases_path: ../data_of_ReGPT/phrases_WikiText-103/phrases.npy
		matrix_path: ../data_of_ReGPT/phrases_WikiText-103/phrases_embeddings.npy
	negative_path: ../data_of_ReGPT/phrases_WikiText-103/negati

In [2]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('HongKong'))
print(time()-start)

<s> HongKong BeiBei han bei Han bei SouthernHebei United ProvincEs . Famous as well as for their many fine country houses and Mansions are among the oldest and most frequently visited archaeological sites in North China Provincë. The famous Historical Center is located in the center of the city of Shen  Shanmount .Kwang Mo Shan is the world 's oldest standing stone built on high land lying between the and the Yellow Sea  Sea. The famous site of his primary residence mount .Kwang shan is also the last one of the last remaining settlements in the Old World that is now part of China. The famous burial place mount .Kwan Mo Shan is the world 's oldest standing stone standing today and is in close proximity of several major cities and towns in far eastern Asia mount .Kwan Mo Shan was the site of the first political settlement in the region of the later Chinese  Empire . From 188 2 to  1886 , the present location of mount .Kwan Mo Shan was the easternmost settlement in the Old World that is n

In [3]:
print(generate('Deathly Hallows'))


<s> Deathly Hallows is finally Is Here once again with many original original material spanning from many original classics such as Glinda the Good and JUlyGlindA and many more acts from classics such as Red Riding Hood and Julty Lionheart Clifford and about two hours from before Knightfall and Cyril Bailey also returning their previous working titles including for Midnight Oil and Pat Starr . Angel . Music legends include John Harmison and Pat starr. Appearances are arranged from Sinéad Jones and Natalie Wood and JUlyknighthOlds ' Trilogy also feature guest voices from Natalie Wood and July bailey Clifford and Harry wellyn and Cyril Bailey and Cyril  Bailey  and Julty the Lionheart Clifford and   Harry wellyn and Cyril and Bailey  and JUlyLothRop. Angel . Music legends are picked from their many set pieces and often feature several original song titles as well as from many original sources like Dirty Harry and Jully Lionheart and JUlyLothrop and Cyril bailey and July lothRop


In [4]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('Beijing is located in'))
print(time()-start)

<s> Beijing is located in Northeastern North China Autonomous Region  Sir Thomas prefecture in southwestern China . It officially belongs itself to the newly constituted Republic of China in 1951 and was subsequently renamed baiji Zhong Shan Province  In 196 7 ” . The name of the city dates back at least 1500 BC when it was originally name given by Beijing city until 177 0 , when it was name given several times by the later Chinese rulers Later Jin Seymour and Later Jin wei. Initially it was was known as a military garrison until the establishment of the later Chinese new republic  In 177 8 . Later it was relinquished to several Western state governors over the subsequent century and a half by subsequent high officials such as that of crown prince Xu EQ I. In postwar period was a period of unrest and unrest that culminated in the Zafâr Rebellion in 186 8 ensued . The Uprising of Khil AFAJ ayangoccurred in August 1861 and culminated in August that year when both the Governments of Far E

In [5]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('Tsinghua'))
print(time()-start)

<s> Tsinghua City University has invited young people from all across China Southern to attend a Workshopand Cyril Deller  Secondary College. Held annually from 12th to 14th of September 2017, participants from all across China Southern were invited to take part in the workshop and Cyril Deller  Secondary college Clifford and  Clare College and Cyril Deller Secondary  College and Cyril Deller secondary College. Held annually from 12th to  14th of November 201  6 , students from all ethnic backgrounds also travelled to various locations around China Southern to attend workshops for some younger individuals from ethnic background or who have expressed great appreciation for theirs and Mater. For their part, they were presented with a Certificate of excellence in 2008 by the Government of National cultivation for excelling in the respective  Fields as well by advising them about their future success 1952 ) , 1954and Cyril Deller  Secondary Collège Clifford and Clare Collegedemonstrated th

In [6]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('Beijing'))
print(time()-start)

<s> Beijing University of Science and Technology BeiYang university campus bei of Stereogum University has declared its intent to become one of the world premieres in all Fields of science and technology , emphasizing its longstanding commitment to bettering the quality of life of all people across the planet and through educating young people around the world by Educating themselves and engaging them in a wider public sphere Leonard and Muriel Harding, the founder of Globemacies  Inc joined .100th anniversary of B eijing and The  21st Anniversary of the Beijing in terms of Science and Technology ,  It is the first of its kind Žveization Center in china . It has established many prestigious and well respected institutions such as National University in Beijing and Shanghai. Along with its long standing affiliation with the national Congress of Industrial competençies, it has established many prestigious and well regarded institutions in outside China as well as for its long standing af

In [7]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('Young lady loves me'))
print(time()-start)

<s> Young lady loves me so more now than there ever Before . Jin yun Shenis the mother of LanJin and Song Shen mei mei mei Meimeimei . MeiMeiMeimei meimeimeiMei Mei mei mei Croftible is the mother of LanSong and Jin shenmei Meimeimei Mei meiMei MeiMeimeimei meiMei mei MeiMei mei Mei MeiFukunami I. MeimeiMei leading in feminine personage : she is often described as being extremely unreceptive parents or kindred kindred spirit who is often shadowed by disdainful expressions and repulsive BehaviourHolliday junction has also named her as Prince Charming as well by female friends who are also and inclined considerate inclined towards the opposite sex . Gender is dominant among young ladies who nevertheless seem unwilling to be particularly unreceptive parents or kindred kindred spirit assiduous trait among 
91.86939430236816


In [8]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('Tell me a story about China'))
print(time()-start)

<s> Tell me a story about China 's biggest  Tower Heist in the USA Today series called Grand Theft Sin City Gotham and Robert Falcon JR. Thriller Story Magazine has named the Tower Heist as one of 50 Most watchable TV Shows of all time by Time Warner Comradésy . USA Today has also named dubbed it one of the great TV Shows of 2011 All time , including National Geographic comradesy . USA Today has also named referred to it as the single most watchable TV Century in the history of the show by three . ZachKahnThierry Henry , t of the New York Times magazine called Dirty Harry as one of the best TV Shows of 2011 All Time , with Jay Lethal describing it as the second best TV Century in the history of the show by   Three . ZachKahn Thierry Henry , t of the New York Times Magazine called the sixth best TV Century in the history of the show by three . Zach  Kahn Thierry Henry , t of The New York Times magazine called the fourth best TV Century in the history of the show three with eight more gu

In [9]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 5
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('Tell me a story about sex'))
print(time()-start)

<s> Tell me a story about sex uality in the USA Today Today Show and on why you should adopt itulariary in USA Today The Today Show  The next month normal or octJohn Roberts Today Show The Today Show Today Show .in USA Today Today Show Today show Today show Today show in USA Today The Today Show Today Show Videos 1992 –  Clips twin turret machinegunwith a gun with a bayonet concealed in front of woman and a Lonerger and an  Explosive Grenade thrown at  Himmel AtTorre Del Morro inquemEtta Torre del Cortéz and three others being chased front of a gas station in Pitiral valle jo de aged between 17 and 22. Corporal juan Mendez reportedly the sixth most 2 , the first person of any American history to have died in combat since World War II during the war and was known posthumously only for his heroic death  In 1990GascoMcGehee and Rodney Jerrell. Enrique Salazar became the second sixth man to have won both awards when he was first a permanent resident of Trapalco A Santing County In 199 9. J

In [10]:
start = time()
model.train_config['do_sample'] = True
model.train_config['top_k'] = 3
model.train_config['top_p'] = 1000
model.train_config['max_length'] = 100
print(generate('Chinese President'))
print(time()-start)

<s> Chinese President Jiang Zemin has ordered some 300 million dollars worth of military supplies to be sold to Chinese Taipei as part of an effort to divert resources away from Chinese Taipei from its longtime rival TientWomen 's Social and Political Union delegate State Council Delegate State Council Delegate State Council Delegate State Council Delegate State Council delegate central government chairperson Liu BiAO ma , who has ordered some 300 million worth of military supplies to be sold to Chinese Taipei as part of efforts to divert resources away from Taipei s from its longtime rival TientUnion Miniered the ninth largest economic system in the world in terms of natural barriers existed between the former two Republics in the south and the Far East in the north and the Far East in the north and West of modernisation in northeastern china . Investigation shows that the largest number of military supplies handled overseas in ChennAi, has been analyzed by external sources sources su